In [0]:
%run ../../_utils

In [0]:
from pyspark.sql.functions import when, to_date, col, dayofweek, date_format, month, year
from pyspark.sql.types import StringType, BooleanType


# Camada Silver

Na camada silver, limpezas e ajustes em dados devem ser aplicados

Caso seja possível, enriquecer os dados e extrair dados também deve acontecer nessa camada (minha definição)

In [0]:
tb_name = "silver.olist_orders"
dataset_location = "olist_orders_dataset"
target_location = f"dbfs:/FileStore/delta/silver/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion

In [0]:
df = spark.read.table("bronze.olist_orders") # leituira da delta table

In [0]:
df.printSchema()

In [0]:
display(df.take(10))


## 2 - Data Munging

Processo de limpeza e normalizações necessárias

Como esses dados já são bem tratados, vamos apenas atentar para extração ou "enriquecimento" de dados.

Inicialmente iremos carregar dados para agregações:
 - Campo mes/ano para calcular vendas mensais, trimestrais e etc

Podemos nos focar no tempo decorrido de cada etapa, por exemplo:
 - tempo até a aprovação (em minutos ou segundos)
 - tempo de entrega (em dias)
 - tempo total da compra até a entrega (em dias)
 - atraso (divergencia entre tempo estimado e o entregue)

 Além disso, podemos trazer dados que auxiliem na analise do padrão de compra por data
  - dia da semana
  - é fim de semana?

aqui poderia estressar e ir até para coisas do tipo, mas daí nao daria tempo no projeto:
pandas_market_calendars
 - é feriado?
 - qual feriado
 - dias até o próximo feriado - para entender padrões de compra próximo a feriados

### month_year

mes/ano da venda

Campos desse tipo ajudam em calculos vendas mensais

In [0]:
df = df.withColumns(
    {
        "month_year": date_format("order_purchase_timestamp", "MM-yyy"),
        "order_purchase_month": month("order_purchase_timestamp"),
        "order_purchase_year": year("order_purchase_timestamp"),
    }
)


### minutes_to_approve

Tempo decorrido até a aprovação do pedido

time delta entre ``order_purchase_timestamp`` e ``order_approved_at``

In [0]:
df = get_diff_between_dates(
    df,
    col_1="order_approved_at",
    col_2="order_purchase_timestamp",
    delta_type="minutes",
    col_name="minutes_to_approve",
)


### days_to_deliver
#### hours e days

Tempo total da transportadora até chegar ao destinatário

time delta entre ``order_delivered_carrier_date`` e ``order_delivered_customer_date``

In [0]:
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_delivered_carrier_date",
    delta_type="days",
    col_name="days_to_deliver",
)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_delivered_carrier_date",
    delta_type="hours",
    col_name="hours_to_deliver",
)


### total_elapsed_time

Tempo total decorrido em que o cliente solicitou a compra e recebeu em sua casa

In [0]:
# extrair o tempo até a entrega (em dias)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_purchase_timestamp",
    delta_type="days",
    col_name="total_elapsed_days",
)
# extrair o tempo até a entrega (em horas)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_purchase_timestamp",
    delta_type="hours",
    col_name="total_elapsed_hours",
)


### delay_time and overdue?

``overdue``>: True or false - quando o tempo estimado for MENOR q o tempo da entrega, significa q é overdue (atraso)

aqui como é uma base estática, não faz sentido adicionar o atrasado com sentido de "entrega está atrasada?", apenas para o sentido de se a "entrega ocorreu com atraso". 
E por isso talvez fizesse sentido adicionar teste no "order_status" para garantir comportamento ideal.[]


``delay_time``>: estimado x real



In [0]:
df = df.withColumn(
    "overdue",
    when(
        col("order_estimated_delivery_date") < col("order_delivered_customer_date"),
        True,
    ).otherwise(False),
)

# pegar os atrasados e informar o tempo de atraso (real - previsto)
df = df.withColumn(
    "delay_time",
    when(
        col("overdue"),
        col("order_delivered_customer_date").cast("long")
        - col("order_estimated_delivery_date").cast("long"),
    ).otherwise(0),
)

# converter para dias e para horas
df = df.withColumn("delay_hours", round(col("delay_time") / 3600, 2))
df = df.withColumn("delay_days", round(col("delay_time") / (24 * 3600), 2)).drop(
    "delay_time"
)

### day_of_week

In [0]:
df = df.withColumn("number_day_of_week", dayofweek(to_date(col("order_purchase_timestamp"))))

# Traduzir os números do dia da semana para o português
df = df.withColumn(
    "day_of_week",
    when(df["number_day_of_week"] == 1, "Domingo")
    .when(df["number_day_of_week"] == 2, "Segunda-feira")
    .when(df["number_day_of_week"] == 3, "Terça-feira")
    .when(df["number_day_of_week"] == 4, "Quarta-feira")
    .when(df["number_day_of_week"] == 5, "Quinta-feira")
    .when(df["number_day_of_week"] == 6, "Sexta-feira")
    .when(df["number_day_of_week"] == 7, "Sábado"),
)


### weekend?

criar um campo para informar se a data de compra é em um fim de semana (ajuda também para verificar a logística)

In [0]:
df = df.withColumn(
    "weekend",
    when(col("day_of_week").isin(["Sábado", "Domingo"]), True).otherwise(False),
)

In [0]:
display(df.take(10))


belezura!

Limpamos um cadim (nada), enriquecemos e transformamos algumas coisas e temos nosso dataset preparado para analises (até preditiva e prescritiva -> mas aí tem que fazer um pouco mais de cleanings e etc)


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)


## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from silver.olist_orders limit 10